In [1]:
import os
import numpy as np
import cv2
import random
import pandas as pd
import shutil
from tqdm import tqdm as tqdm

In [2]:
img_path = '/Users/mryu/Desktop/goss_img_all/'
out_img_path = '/Users/mryu/Desktop/img/'

In [3]:
img_name_ls = os.listdir(img_path)
# img_ls

# 打乱顺序划分出 训练集、验证集和测试集（8:1:1）

In [4]:
ratio1 = 0.8
ratio2 = 0.9
img_num = len(img_name_ls)
offset1 = int(img_num * ratio1)
offset2 = int(img_num * ratio2)

random.shuffle(img_name_ls)
img_name_train_ls = img_name_ls[:offset1]
img_name_dev_ls = img_name_ls[offset1:offset2]
img_name_test_ls = img_name_ls[offset2:]

# len(img_name_train_ls), len(img_name_dev_ls), len(img_name_test_ls)

# 依照划分的训练集、验证集和测试集划分出对应的 csv 文件

In [5]:
data_csv = pd.read_csv('/Users/mryu/Desktop/gossipcop_multi.csv')
# data_csv.loc[723, :]
# 'gossipcop-832300074' in data_csv['img_id'].values

In [6]:
flag = 0
df_train_index_ls = []
df_dev_index_ls = []
df_test_index_ls = []

for dataset in [img_name_train_ls, img_name_dev_ls, img_name_test_ls]:
#     print(len(dataset))
    flag = flag + 1 
    for img_name in dataset:
        img_id_str = img_name[:-4]
        
        if img_id_str in data_csv['img_id'].values and flag == 1:
            df_train_index = data_csv[data_csv.img_id == img_id_str].index.tolist()[0]
            df_train_index_ls.append(df_train_index)
            
        if img_id_str in data_csv['img_id'].values and flag == 2:
            df_dev_index = data_csv[data_csv.img_id == img_id_str].index.tolist()[0]
            df_dev_index_ls.append(df_dev_index)
        
        if img_id_str in data_csv['img_id'].values and flag == 3:
            df_test_index = data_csv[data_csv.img_id == img_id_str].index.tolist()[0]  
            df_test_index_ls.append(df_test_index)

df_train = data_csv.loc[df_train_index_ls, :]
df_dev = data_csv.loc[df_dev_index_ls, :]
df_test = data_csv.loc[df_test_index_ls, :]

df_train.to_csv('/Users/mryu/Desktop/train.csv')
df_dev.to_csv('/Users/mryu/Desktop/dev.csv')
df_test.to_csv('/Users/mryu/Desktop/test.csv')

print(' <----------- DONE -----------> ')

len(df_train), len(df_dev), len(df_test)
# df_train.head()

 <----------- DONE -----------> 


(5445, 683, 677)

# 将图片按照种类分开

In [12]:
train_image_id_list = list(np.asarray(df_train['img_id']))
dev_image_id_list = list(np.asarray(df_dev['img_id']))
test_image_id_list = list(np.asarray(df_test['img_id']))
# print(train_image_id_list)
print('train_len: ', len(train_image_id_list))
print('dev_len: ', len(dev_image_id_list))
print('test_len: ', len(test_image_id_list))

path_images = '/Users/mryu/Desktop/goss_img_all'
image_ls = os.listdir(path_images)
# print(len(image_ls))
# print(image_ls[0])


for image in image_ls:
    image_id = image.replace('.jpg', '')
    
    try:
        if image_id in train_image_id_list:
            shutil.move('/Users/mryu/Desktop/goss_img_all/' + image,
                        '/Users/mryu/Desktop/train_img')
#             print('in trian!')

        elif image_id in dev_image_id_list:
            shutil.move('/Users/mryu/Desktop/goss_img_all/' + image,
                        '/Users/mryu/Desktop/dev_img')
#             print('in dev!')
            
        elif image_id in test_image_id_list:
            shutil.move('/Users/mryu/Desktop/goss_img_all/' + image,
                        '/Users/mryu/Desktop/test_img')
        
#             print('in test!')

    
    except:
        
        print('Wrong: ', image)
        
        

print(' <----------- DONE -----------> ')

['gossipcop-914193', 'gossipcop-914670', 'gossipcop-4769457532', 'gossipcop-915701', 'gossipcop-903697', 'gossipcop-918513', 'gossipcop-2689700497', 'gossipcop-953283', 'gossipcop-942603', 'gossipcop-867491', 'gossipcop-889619', 'gossipcop-5716325201', 'gossipcop-951645', 'gossipcop-902831', 'gossipcop-875983', 'gossipcop-912135', 'gossipcop-9160143371', 'gossipcop-917638', 'gossipcop-897266', 'gossipcop-4461739638', 'gossipcop-921538', 'gossipcop-3430736744', 'gossipcop-870845', 'gossipcop-4314683692', 'gossipcop-892353', 'gossipcop-841974', 'gossipcop-8975289821', 'gossipcop-900275', 'gossipcop-882603', 'gossipcop-846511', 'gossipcop-8865743828', 'gossipcop-1658391416', 'gossipcop-857993', 'gossipcop-3393009926', 'gossipcop-845722', 'gossipcop-892317', 'gossipcop-2796368055', 'gossipcop-863571', 'gossipcop-875934', 'gossipcop-867070', 'gossipcop-904339', 'gossipcop-2715725198', 'gossipcop-1999047183', 'gossipcop-5835779872', 'gossipcop-6537881170', 'gossipcop-860430', 'gossipcop-3435

# 图片保存为 npy，调整大小为 （224， 224）

In [27]:
def save_npy(img_id_ls, img_folder_name, type_npy):

    img_ls = []
    flag = 0
    
    base_img_path = '/Users/mryu/Desktop/'

    for img_id in img_id_ls:
        img = cv2.imread(base_img_path + img_folder_name + '/' + img_id + '.jpg')
        try:
            b, g , r = cv2.split(img)  # 分割出三个通道
            img = cv2.merge([r, g ,b])
            img=cv2.resize(img, (224,224), interpolation=cv2.INTER_CUBIC)
            img_ls.append(img)

        except:
    #         os.remove(img_path + img_name)
            flag  = flag + 1

    img_npy = np.array(img_ls)
    np.save('/Users/mryu/Desktop/img_npy/' + type_npy, img_npy)
    print(flag)
    print(' <-------- ', type_npy, 'DONE -------->')

In [33]:
save_npy(train_image_id_list, 'train_img', 'train_img')
save_npy(dev_image_id_list, 'dev_img', 'dev_img')
save_npy(test_image_id_list, 'test_img', 'test_img')

0
 <--------  train_img DONE -------->
0
 <--------  dev_img DONE -------->
0
 <--------  test_img DONE -------->


# 读取 npy 测试一下

In [32]:
train_img_npy = np.load('/Users/mryu/Desktop/img_npy/train_img.npy')
dev_img_npy = np.load('/Users/mryu/Desktop/img_npy/dev_img.npy')
test_img_npy = np.load('/Users/mryu/Desktop/img_npy/test_img.npy')
train_img_npy.shape, dev_img_npy.shape, test_img_npy.shape

((5445, 224, 224, 3), (683, 224, 224, 3), (677, 224, 224, 3))